In [ ]:
import pandas as pd
from datetime import timedelta

In [ ]:
data = pd.read_csv('mimic raw data.csv')
data

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,10056037,30687961,F,68,2131/3/30 22:00,127.0,NaN,NaN,NaN,31.3,NaN
1,10056037,30687961,F,68,2131/3/30 22:19,115.0,NaN,NaN,42.0,NaN,NaN
2,10056037,30687961,F,68,2131/3/30 22:20,114.0,NaN,NaN,30.0,NaN,NaN
3,10056037,30687961,F,68,2131/3/30 22:22,NaN,NaN,NaN,NaN,NaN,100.0
4,10056037,30687961,F,68,2131/3/30 22:23,111.0,NaN,NaN,17.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
148329,19993336,36483324,F,81,2171/9/29 14:01,NaN,104.0,55.0,NaN,NaN,NaN
148330,19993336,36483324,F,81,2171/9/29 15:00,85.0,NaN,NaN,26.0,NaN,94.0
148331,19993336,36483324,F,81,2171/9/29 15:01,NaN,99.0,55.0,NaN,NaN,NaN
148332,19993336,36483324,F,81,2171/9/29 16:00,86.0,NaN,NaN,20.0,NaN,95.0


In [ ]:
# 去除小於20歲和大於200歲的患者
data = data[data["anchor_age"] > 20]
data = data[data["anchor_age"] < 200]
data

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,10056037,30687961,F,68,2131/3/30 22:00,127.0,NaN,NaN,NaN,31.3,NaN
1,10056037,30687961,F,68,2131/3/30 22:19,115.0,NaN,NaN,42.0,NaN,NaN
2,10056037,30687961,F,68,2131/3/30 22:20,114.0,NaN,NaN,30.0,NaN,NaN
3,10056037,30687961,F,68,2131/3/30 22:22,NaN,NaN,NaN,NaN,NaN,100.0
4,10056037,30687961,F,68,2131/3/30 22:23,111.0,NaN,NaN,17.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
148329,19993336,36483324,F,81,2171/9/29 14:01,NaN,104.0,55.0,NaN,NaN,NaN
148330,19993336,36483324,F,81,2171/9/29 15:00,85.0,NaN,NaN,26.0,NaN,94.0
148331,19993336,36483324,F,81,2171/9/29 15:01,NaN,99.0,55.0,NaN,NaN,NaN
148332,19993336,36483324,F,81,2171/9/29 16:00,86.0,NaN,NaN,20.0,NaN,95.0


In [ ]:
# 匯入每位患者CA的時間資料
time = pd.read_csv('CA time.csv')
time

,subject_id,stay_id,starttime
0,11106524,37574043,2190/5/31 03:14
1,11106897,38083473,2172/8/10 10:45
2,11129835,36471889,2154/7/18 05:35
3,11129835,36471889,2154/7/18 04:40
4,11134366,31912411,2157/10/9 04:45
...,...,...,...
570,11022710,33464702,2112/3/6 09:15
571,11058564,36505769,2111/11/11 08:10
572,11061304,30601889,2161/6/19 10:10
573,19754383,35683133,2163/6/2 11:21


In [ ]:
print(time['subject_id'].nunique())
print(time['stay_id'].nunique())
print(time['starttime'].nunique())

501
503
574


In [ ]:
# 將CA時間進行排序(早到晚)
time.sort_values(['starttime','stay_id'], inplace=True)
time

,subject_id,stay_id,starttime
315,16504173,35859388,2110/7/12 10:15
528,10352416,35043893,2110/9/11 16:15
529,10352416,35043893,2110/9/11 20:10
530,10352416,35043893,2110/9/11 21:19
571,11058564,36505769,2111/11/11 08:10
...,...,...,...
109,12682640,30515444,2198/7/3 18:18
500,19538920,30183853,2201/8/12 19:20
514,10152414,34547307,2203/7/5 06:40
80,12426774,30308646,2204/2/15 18:00


In [ ]:
# 將同一次住院發生二次以上驟停的資料視為True
mask = time.groupby('stay_id').cumcount() > 0
mask

315    False
528    False
529     True
530     True
571    False
       ...  
109    False
500    False
514    False
80     False
189    False
Length: 575, dtype: bool

In [ ]:
# 只留下第一次的資料
firstevent = time[~mask]
firstevent

,subject_id,stay_id,starttime
315,16504173,35859388,2110/7/12 10:15
528,10352416,35043893,2110/9/11 16:15
571,11058564,36505769,2111/11/11 08:10
84,12342976,31548284,2111/2/11 13:47
7,11191438,39793956,2112/1/3 16:40
...,...,...,...
109,12682640,30515444,2198/7/3 18:18
500,19538920,30183853,2201/8/12 19:20
514,10152414,34547307,2203/7/5 06:40
80,12426774,30308646,2204/2/15 18:00


In [ ]:
# 將患者和驟停資料的stay_id視為index
data = data.set_index('stay_id')
firstevent = firstevent.set_index('stay_id')

In [ ]:
# 將兩個表格合併，只在表格內新增第一次CA時間，並沒有對vital sign進行資料清理
merged_data = pd.merge(data, firstevent[['starttime']], left_index=True, right_index=True, how='inner')
merged_data

,subject_id,gender,anchor_age,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2,starttime
stay_id,,,,,,,,,,,
30031755,19867017,F,74,2124/4/19 18:07,114.0,NaN,NaN,18.0,NaN,NaN,2124/4/24 17:00
30031755,19867017,F,74,2124/4/19 18:18,NaN,NaN,NaN,NaN,NaN,96.0,2124/4/24 17:00
30031755,19867017,F,74,2124/4/19 18:32,108.0,NaN,NaN,20.0,NaN,95.0,2124/4/24 17:00
30031755,19867017,F,74,2124/4/19 20:00,111.0,NaN,NaN,18.0,NaN,96.0,2124/4/24 17:00
30031755,19867017,F,74,2124/4/19 20:15,NaN,85.0,60.0,NaN,NaN,NaN,2124/4/24 17:00
...,...,...,...,...,...,...,...,...,...,...,...
39992578,19774838,M,69,2187/9/16 10:00,80.0,NaN,NaN,24.0,NaN,99.0,2187/9/15 10:00
39992578,19774838,M,69,2187/9/16 11:00,80.0,NaN,NaN,24.0,NaN,98.0,2187/9/15 10:00
39992578,19774838,M,69,2187/9/16 12:00,80.0,NaN,NaN,24.0,37.0,99.0,2187/9/15 10:00


In [ ]:
merged_data = merged_data.reset_index()
merged_data

,stay_id,subject_id,gender,anchor_age,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2,starttime
0,30031755,19867017,F,74,2124/4/19 18:07,114.0,NaN,NaN,18.0,NaN,NaN,2124/4/24 17:00
1,30031755,19867017,F,74,2124/4/19 18:18,NaN,NaN,NaN,NaN,NaN,96.0,2124/4/24 17:00
2,30031755,19867017,F,74,2124/4/19 18:32,108.0,NaN,NaN,20.0,NaN,95.0,2124/4/24 17:00
3,30031755,19867017,F,74,2124/4/19 20:00,111.0,NaN,NaN,18.0,NaN,96.0,2124/4/24 17:00
4,30031755,19867017,F,74,2124/4/19 20:15,NaN,85.0,60.0,NaN,NaN,NaN,2124/4/24 17:00
...,...,...,...,...,...,...,...,...,...,...,...,...
141919,39992578,19774838,M,69,2187/9/16 10:00,80.0,NaN,NaN,24.0,NaN,99.0,2187/9/15 10:00
141920,39992578,19774838,M,69,2187/9/16 11:00,80.0,NaN,NaN,24.0,NaN,98.0,2187/9/15 10:00
141921,39992578,19774838,M,69,2187/9/16 12:00,80.0,NaN,NaN,24.0,37.0,99.0,2187/9/15 10:00
141922,39992578,19774838,M,69,2187/9/16 13:00,80.0,NaN,NaN,24.0,NaN,100.0,2187/9/15 10:00


In [ ]:
# 重新調整欄位順序
merged_data = merged_data.reindex(columns=['stay_id','subject_id','gender','anchor_age','starttime','charttime', 'heart_rate', 'sbp_ni','dbp_ni','resp_rate','temperature','spo2'])
merged_data

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124/4/24 17:00,2124/4/19 18:07,114.0,NaN,NaN,18.0,NaN,NaN
1,30031755,19867017,F,74,2124/4/24 17:00,2124/4/19 18:18,NaN,NaN,NaN,NaN,NaN,96.0
2,30031755,19867017,F,74,2124/4/24 17:00,2124/4/19 18:32,108.0,NaN,NaN,20.0,NaN,95.0
3,30031755,19867017,F,74,2124/4/24 17:00,2124/4/19 20:00,111.0,NaN,NaN,18.0,NaN,96.0
4,30031755,19867017,F,74,2124/4/24 17:00,2124/4/19 20:15,NaN,85.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
141919,39992578,19774838,M,69,2187/9/15 10:00,2187/9/16 10:00,80.0,NaN,NaN,24.0,NaN,99.0
141920,39992578,19774838,M,69,2187/9/15 10:00,2187/9/16 11:00,80.0,NaN,NaN,24.0,NaN,98.0
141921,39992578,19774838,M,69,2187/9/15 10:00,2187/9/16 12:00,80.0,NaN,NaN,24.0,37.0,99.0
141922,39992578,19774838,M,69,2187/9/15 10:00,2187/9/16 13:00,80.0,NaN,NaN,24.0,NaN,100.0


In [ ]:
# 將starttime和charttime欄位轉成datetime格式
merged_data['starttime'] = pd.to_datetime(merged_data['starttime'])
merged_data['charttime'] = pd.to_datetime(merged_data['charttime'])

In [ ]:
# 將發生第一次CA後的資料刪除
mask1 = merged_data["charttime"] <= merged_data["starttime"]
df = merged_data[mask1]
df

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,NaN,NaN,18.0,NaN,NaN
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,NaN,NaN,NaN,NaN,NaN,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,NaN,NaN,20.0,NaN,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,NaN,NaN,18.0,NaN,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,NaN,85.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
141889,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,NaN,NaN,11.0,NaN,99.0
141890,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,NaN,NaN,16.0,NaN,99.0
141891,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,NaN,NaN,18.0,37.833333,99.0
141892,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,NaN,NaN,22.0,NaN,98.0


In [ ]:
# 移除charttime是NA的row
# 去除六個生命徵象都是NA的row

df = df.dropna(subset=['charttime'])
df = df.dropna(how='all', subset=['heart_rate', 'sbp_ni','dbp_ni','resp_rate','temperature','spo2'])
df

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,NaN,NaN,18.0,NaN,NaN
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,NaN,NaN,NaN,NaN,NaN,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,NaN,NaN,20.0,NaN,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,NaN,NaN,18.0,NaN,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,NaN,85.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
141889,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,NaN,NaN,11.0,NaN,99.0
141890,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,NaN,NaN,16.0,NaN,99.0
141891,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,NaN,NaN,18.0,37.833333,99.0
141892,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,NaN,NaN,22.0,NaN,98.0


In [ ]:
print(df[['subject_id', 'stay_id', 'starttime']].nunique())

subject_id    477
stay_id       479
starttime     479
dtype: int64


In [ ]:
# 匯出csv，raw data for demographics
df.to_csv('mimic CA demo.csv', index=False)

In [ ]:
# 確認資料中是否有重複時間的
duplicate_rows = df[df.duplicated(['stay_id', 'charttime'], keep=False)].index
df.loc[duplicate_rows]

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2


In [ ]:
# 去除outlier
df['sbp_ni'] = df['sbp_ni'].apply(lambda x: x if 0 < x < 400 else None)
df['dbp_ni'] = df['dbp_ni'].apply(lambda x: x if 0 < x < 300 else None)
df['heart_rate'] = df['heart_rate'].apply(lambda x: x if 0 < x < 300 else None)
df['resp_rate'] = df['resp_rate'].apply(lambda x: x if 0 < x < 70 else None)
df['temperature'] = df['temperature'].apply(lambda x: x if 10 < x < 50 else None)
df['spo2'] = df['spo2'].apply(lambda x: x if 0 < x <= 100 else None)

df

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,NaN,NaN,18.0,NaN,NaN
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,NaN,NaN,NaN,NaN,NaN,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,NaN,NaN,20.0,NaN,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,NaN,NaN,18.0,NaN,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,NaN,85.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
141889,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,NaN,NaN,11.0,NaN,99.0
141890,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,NaN,NaN,16.0,NaN,99.0
141891,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,NaN,NaN,18.0,37.833333,99.0
141892,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,NaN,NaN,22.0,NaN,98.0


In [ ]:
# 進行線性內插法
interpolate_df = df.interpolate(limit_direction = 'both')
interpolate_df

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,85.0,60.0,18.0,37.333333,96.0
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,111.0,85.0,60.0,19.0,37.333333,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,85.0,60.0,20.0,37.333333,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,85.0,60.0,18.0,37.333333,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,111.0,85.0,60.0,22.0,37.333333,95.5
...,...,...,...,...,...,...,...,...,...,...,...,...
141889,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,88.0,33.0,11.0,37.537037,99.0
141890,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,88.0,33.0,16.0,37.685185,99.0
141891,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,88.0,33.0,18.0,37.833333,99.0
141892,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,88.0,33.0,22.0,37.833333,98.0


In [ ]:
interpolate_df.head(10)

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.000000,85.0,60.0,18.000000,37.333333,96.0
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,111.000000,85.0,60.0,19.000000,37.333333,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.000000,85.0,60.0,20.000000,37.333333,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.000000,85.0,60.0,18.000000,37.333333,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,111.000000,85.0,60.0,22.000000,37.333333,95.5
5,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 21:00:00,111.000000,85.0,53.0,26.000000,37.333333,95.0
6,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 21:14:00,108.000000,81.0,50.0,24.000000,37.333333,95.0
7,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 22:00:00,108.000000,77.0,47.0,26.000000,37.342593,95.0
8,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 23:00:00,108.000000,83.0,44.0,18.000000,37.351852,95.0
9,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 23:15:00,109.666667,89.0,41.0,21.333333,37.361111,95.0


In [ ]:
# 四捨五入
interpolate_df['heart_rate'] = interpolate_df['heart_rate'].apply(lambda x:round(x,0))
interpolate_df['sbp_ni'] = interpolate_df['sbp_ni'].apply(lambda x:round(x,0))
interpolate_df['dbp_ni'] = interpolate_df['dbp_ni'].apply(lambda x:round(x,0))
interpolate_df['resp_rate'] = interpolate_df['resp_rate'].apply(lambda x:round(x,0))
interpolate_df['temperature'] = interpolate_df['temperature'].apply(lambda x:round(x,1))
interpolate_df['spo2'] = interpolate_df['spo2'].apply(lambda x:round(x,0))

In [ ]:
interpolate_df.head(10)

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,85.0,60.0,18.0,37.3,96.0
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,111.0,85.0,60.0,19.0,37.3,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,85.0,60.0,20.0,37.3,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,85.0,60.0,18.0,37.3,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,111.0,85.0,60.0,22.0,37.3,96.0
5,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 21:00:00,111.0,85.0,53.0,26.0,37.3,95.0
6,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 21:14:00,108.0,81.0,50.0,24.0,37.3,95.0
7,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 22:00:00,108.0,77.0,47.0,26.0,37.3,95.0
8,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 23:00:00,108.0,83.0,44.0,18.0,37.4,95.0
9,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 23:15:00,110.0,89.0,41.0,21.0,37.4,95.0


In [ ]:
# 清理後的CA患者數量
print(interpolate_df[['subject_id', 'stay_id', 'starttime']].nunique())

subject_id    477
stay_id       479
starttime     479
dtype: int64


In [ ]:
# 前處理後的資料，for model
interpolate_df.to_csv('mimic CA.csv', index=False)